In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [40]:
dataEvents = pd.read_csv('/Users/manuelgomezmoratilla/Desktop/TFG/data_processing_scripts/data/anonamyze_all_data_collection_v2.csv', sep=";")

In [42]:
userFunnelByPuzzle = computeFunnelByPuzzle(dataEvents)

In [48]:
pd.options.display.max_colwidth = 100

In [41]:
def computeFunnelByPuzzle(dataEvents, group = 'all'):
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    
    # userFunnelDict key: (group~user~puzzle), json values: started, create_shape, submitted, completed
    userFunnelDict = dict()   

    for user in dataEvents['group_user_id'].unique():

        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_puzzle_key = None
        
        for enum, event in user_events.iterrows():

            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):
                user_puzzle_key = event['group'] + '~' + event['user'] + '~' + json.loads(event['data'])['task_id']
                if(user_puzzle_key not in userFunnelDict.keys()):
                    userFunnelDict[user_puzzle_key] = json.loads('{"started": 0, "create_shape": 0, "submitted": 0, "completed": 0, "total": 0}')

            if(event['type'] == 'ws-puzzle_started'):
                userFunnelDict[user_puzzle_key]['started'] += 1
                userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-create_shape'):
                userFunnelDict[user_puzzle_key]['create_shape'] += 1
                userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-check_solution'):
                userFunnelDict[user_puzzle_key]['submitted'] += 1
                userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-puzzle_complete'):
                userFunnelDict[user_puzzle_key]['completed'] += 1
                userFunnelDict[user_puzzle_key]['total'] += 1
                   
    userFunnelList = []
    for key in userFunnelDict.keys():
        key_split = key.split('~')
        userFunnelList.append([key_split[0], key_split[1], key_split[2], userFunnelDict[key]])
        
    userFunnelByPuzzle = pd.DataFrame(userFunnelList, columns=['group', 'user', 'task_id', 'funnel'])
    
    
    return userFunnelByPuzzle

In [37]:
def computeFunnelByUser(dataEvents, group = 'all'):
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    
    # userFunnelDict key: (group~user~puzzle), json values: started, create_shape, submitted, completed
    userFunnelDict = dict()   

    for user in dataEvents['group_user_id'].unique():

        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_puzzle_key = None
        
        for enum, event in user_events.iterrows():

            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):
                user_puzzle_key = event['group'] + '~' + event['user'] + '~' + json.loads(event['data'])['task_id']
                if(user_puzzle_key not in userFunnelDict.keys()):
                    userFunnelDict[user_puzzle_key] = json.loads('{"started": 0, "create_shape": 0, "submitted": 0, "completed": 0}')

            if(event['type'] == 'ws-puzzle_started'):
                userFunnelDict[user_puzzle_key]['started'] = 1
                #userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-create_shape'):
                userFunnelDict[user_puzzle_key]['create_shape'] = 1
                #userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-check_solution'):
                userFunnelDict[user_puzzle_key]['submitted'] = 1
                #userFunnelDict[user_puzzle_key]['total'] += 1
            elif(event['type'] == 'ws-puzzle_complete'):
                userFunnelDict[user_puzzle_key]['completed'] = 1
                #userFunnelDict[user_puzzle_key]['total'] += 1
    
    userFunnelList = []
    for key in userFunnelDict.keys():
        key_split = key.split('~')
        #key_2 = key_split[0] + '~' + key_split[1]
        if (key_split[2] != 'Sandbox') : 
            userFunnelList.append([key_split[0], key_split[1], key_split[2], userFunnelDict[key]['started'], userFunnelDict[key]['create_shape'], userFunnelDict[key]['submitted'], userFunnelDict[key]['completed']])
        
    userFunnelByPuzzle = pd.DataFrame(userFunnelList, columns=['group', 'user', 'task_id', 'started', 'create_shape', 'submitted', 'completed'])
    userFunnelByUser = userFunnelByPuzzle.groupby(['user', 'group']).agg({'task_id' : 'count', 'started' : 'sum', 'create_shape' : 'sum', 'submitted' : 'sum', 'completed' : 'sum'})
    
    return userFunnelByUser

In [38]:
userFunnelByUser = computeFunnelByUser(dataEvents)
userFunnelByUser

,,task_id,started,create_shape,submitted,completed
user,group,,,,,
00f86c5fc28e8d326cdae17ea97c7387,e6af7d42084352a39449e6d0a09b18cd,28,28,27,26,16
0157bc278747755031338f18c5ba4233,4fe25833f555e9903d2bb6bbeec3fbfb,8,8,8,8,7
031395e5e53450933d46c8646ae4ee27,e6af7d42084352a39449e6d0a09b18cd,7,7,7,6,5
049217092f0cf180df6272f85c8144b1,e6af7d42084352a39449e6d0a09b18cd,30,30,28,27,24
04efb31113020a35abc57f18b7c92b34,e6af7d42084352a39449e6d0a09b18cd,28,28,28,28,27
...,...,...,...,...,...,...
fd8558801677dd29161b880a74cb9c81,8cbfa61cb2b025b16b04a8e470422960,29,29,29,28,28
fd934ef7ae0a2bc9ef543c6317cd3fd0,e6af7d42084352a39449e6d0a09b18cd,2,2,2,2,1
ff8662d59180b49cef99ccbea2414fac,7a01ab37bd2729626dcaf96166390315,30,30,26,26,23


In [39]:
userFunnelByUser.to_csv("funnelOutput.csv", decimal = ",")